In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('/content/drive/My Drive/Copy of data.csv')
df.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1, inplace=True)
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1


Оставим только категориальные данные, от impressions избавимся. Последний день в тест, предпоследний выделим для валидации.

In [4]:
df['oaid_hash'].value_counts()

308174966294367527     5243
2890718152668627077    2511
2521895603443866206    2289
8212556321845734673    1974
3375698397737628939    1959
                       ... 
7989197060799678922       1
3207898120952090573       1
8578082337233412909       1
8779980718415233023       1
453968700792456599        1
Name: oaid_hash, Length: 6510316, dtype: int64

Видно, что присутствуют редки пользователи, но не будем их трогать

In [7]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
  df.drop(['campaign_clicks', 'impressions'], axis=1, inplace=True)
  datetime = pd.to_datetime(df['date_time'])
  df['date'] = datetime.dt.date
  df.drop('date_time', axis=1, inplace=True)

  df_train = df[df['date'] < pd.Timestamp('2021-10-01').date()]
  df_train.drop('date', axis=1, inplace=True)
  # X_train = df_train.drop(['clicks', 'date'], axis=1)
  # y_train = df_train['clicks']

  df_val = df[df['date'] == pd.Timestamp('2021-10-01').date()]
  df_val.drop('date', axis=1, inplace=True)
  # X_val = df_val.drop(['clicks', 'date'], axis=1)
  # y_val = df_val['clicks']

  df_test = df[df['date'] == pd.Timestamp('2021-10-02').date()]
  df_test.drop('date', axis=1, inplace=True)
  # X_test = df_test.drop(['clicks', 'date'], axis=1)
  # y_test = df_test['clicks']

  return df_train, df_val, df_test

In [8]:
df_train, df_val, df_test = feature_engineering(df)


<ipython-input-7-5d3f884a2840>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop('date', axis=1, inplace=True)
<ipython-input-7-5d3f884a2840>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.drop('date', axis=1, inplace=True)
<ipython-input-7-5d3f884a2840>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop('date', axis=1, inplace=True)


Приведем данные в нужный формат для обучения

In [13]:
#  https://github.com/Bobe24/Dataframe2libffm/blob/master/dataframe2libffm.py
category_column = [
  'zone_id',
  'banner_id',
  'os_id',
  'country_id',
  'clicks',
  'oaid_hash'
  ]

class FFMFormat:
    def __init__(self):
        self.field_index_ = None
        self.feature_index_ = None
        self.y = None

    def fit(self, df, y=None):
        self.y = y
        df_ffm = df[df.columns.difference([self.y])]
        if self.field_index_ is None:
            self.field_index_ = {col: i for i, col in enumerate(df_ffm)}

        if self.feature_index_ is not None:
            last_idx = max(list(self.feature_index_.values()))

        if self.feature_index_ is None:
            self.feature_index_ = dict()
            last_idx = 0

        for col in df_ffm.columns:
            vals = df_ffm[col].unique()
            for val in vals:
                if pd.isnull(val):
                    continue
                name = '{}_{}'.format(col, val)
                if name not in self.feature_index_:
                    self.feature_index_[name] = last_idx
                    last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row, t):
        ffm = []
        if self.y != None:
            ffm.append(str(row.loc[row.index == self.y][0]))
        if self.y is None:
            ffm.append(str(0))

        for col, val in row.loc[row.index != self.y].to_dict().items():
            col_type = t[col]
            name = '{}_{}'.format(col, val)
            # if col_type.kind == 'O':
            if col in category_column:
                ffm.append('{}:{}:1'.format(self.field_index_[col],
                                            self.feature_index_[name]))
            else:
            # elif col_type.kind == 'i':
                ffm.append('{}:{}:{}'.format(self.field_index_[col],
                                             self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        t = df.dtypes.to_dict()
        return pd.Series(
            {idx: self.transform_row_(row, t) for idx, row in df.iterrows()})


In [14]:
ffm_encoder = FFMFormat()
ffm_train = ffm_encoder.fit_transform(df_train, y='clicks')
ffm_train.to_csv('train.txt', index=False, header=False)

ffm_val = ffm_encoder.fit_transform(df_val, y='clicks')
ffm_val.to_csv('val.txt', index=False, header=False)

In [16]:
!cp val.txt /content/drive/MyDrive/recsys_hw2

Для обучения модели будем использовать библиотеку xlearn

In [1]:
!pip install xlearn

In [1]:
import xlearn as xl
from sklearn.model_selection import ParameterGrid

In [2]:
import os
os.environ['USER'] = 'test'

In [3]:
param_grid = {
    'task': ['binary'],
    'lr': [0.1],
    'lambda':[0.1, 0.001, 0.0001],
    'k': [4, 8],
    'metric': ['auc'],
}
for param in ParameterGrid(param_grid):
  print(param)
  ffm_model = xl.create_ffm()
  ffm_model.setTrain('train.txt')
  ffm_model.setValidate('val.txt')
  ffm_model.fit(param, 'model.out')

{'k': 4, 'lambda': 0.1, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
{'k': 4, 'lambda': 0.001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
{'k': 4, 'lambda': 0.0001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
{'k': 8, 'lambda': 0.1, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
{'k': 8, 'lambda': 0.001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}
{'k': 8, 'lambda': 0.0001, 'lr': 0.1, 'metric': 'auc', 'task': 'binary'}


Лучшие результаты оказались у модели .... Проверим на тестовых данных.

In [ ]:
ffm_encoder = FFMFormat()
ffm_test = ffm_encoder.fit_test(df_train, y='clicks')
ffm_test.to_csv('test.txt', index=False, header=False)